In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [2]:
all_player =  pd.read_csv("all_seasons.csv", index_col="Rk")
all_player = all_player[['player_name', 'college']]
all_player.rename(columns={'player_name': 'Player'}, inplace=True)
all_player = all_player.drop_duplicates(subset=['Player'])
contract = pd.read_csv("sportsref_download.csv", index_col="Rk")
contract = contract[['Player', 'Tm', '2023-24', 'Guaranteed']]

In [3]:
main_data = pd.merge(all_player, contract, on='Player')
main_data = main_data.drop_duplicates(subset=['Player'])